# Context-sensitive Spelling Correction

The goal of the assignment is to implement context-sensitive spelling correction. The input of the code will be a set of text lines and the output will be the same lines with spelling mistakes fixed.

Submit the solution of the assignment to Moodle as a link to your GitHub repository containing this notebook.

Useful links:
- [Norvig's solution](https://norvig.com/spell-correct.html)
- [Norvig's dataset](https://norvig.com/big.txt)
- [Ngrams data](https://www.ngrams.info/download_coca.asp)

Grading:
- 60 points - Implement spelling correction
- 20 points - Justify your decisions
- 20 points - Evaluate on a test set


## Implement context-sensitive spelling correction

Your task is to implement context-sensitive spelling corrector using N-gram language model. The idea is to compute conditional probabilities of possible correction options. For example, the phrase "dking sport" should be fixed as "doing sport" not "dying sport", while "dking species" -- as "dying species".

The best way to start is to analyze [Norvig's solution](https://norvig.com/spell-correct.html) and [N-gram Language Models](https://web.stanford.edu/~jurafsky/slp3/3.pdf).

You may also want to implement:
- spell-checking for a concrete language - Russian, Tatar, etc. - any one you know, such that the solution accounts for language specifics,
- some recent (or not very recent) paper on this topic,
- solution which takes into account keyboard layout and associated misspellings,
- efficiency improvement to make the solution faster,
- any other idea of yours to improve the Norvig’s solution.

IMPORTANT:  
Your project should not be a mere code copy-paste from somewhere. You must provide:
- Your implementation
- Analysis of why the implemented approach is suggested
- Improvements of the original approach that you have chosen to implement

In [210]:
import pandas as pd
import nltk
import math
nltk.download('punkt')  # Download necessary resources (if not already downloaded)

from nltk.tokenize import sent_tokenize, word_tokenize

class SpellChecker:
  def __init__(self, WORDS_FREQS, BIGRAMS, edit_distance=2, beam_search_N=20):
    self.WORDS_FREQS = WORDS_FREQS
    self.N = sum(WORDS_FREQS.values())
    self.N_bigrams = sum(BIGRAMS.values())
    self.BIGRAMS = BIGRAMS
    self.edit_distance = edit_distance
    self.beam_search_N = beam_search_N

  def p(self, word):
    if word.lower() not in self.WORDS_FREQS:
      return 1
    return self.WORDS_FREQS[word.lower()] / self.N

  def two_word_seq_prob(self, word1, word2):
    if (word1, word2) in self.BIGRAMS:
      return self.BIGRAMS[(word1, word2)] / self.N_bigrams
    else:
      return self.p(word1) * self.p(word2)

  def existed_word_neighbors(self, word):
    existed_words = self.known([word.lower()])
    current_edit_words = set([word.lower()])
    for i in range(self.edit_distance):
      current_words = set()
      for word in current_edit_words:
        current_words |= self.edits1(word)
      existed_words += sorted(self.known(current_words), key=lambda x: -self.p(x))
      current_edit_words = current_words
    uniq_words = set()
    ans = []
    for word in existed_words:
      if word not in uniq_words:
        uniq_words.add(word)
        ans.append(word)
        if len(ans) == self.beam_search_N:
          break
    return ans
  def edits1(self, word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

  def known(self, words):
     return [w for w in words if w in self.WORDS_FREQS]

  def tokenize(self, text):
    return [word_tokenize(sentence) for sentence in sent_tokenize(text)]
  def prob_lighting(self, p):

    return 2 / (1 + math.exp(-10 * p)) - 1
  def sentence_correction(self, sentence):
    if len(sentence) < 3: # One word sentence
      candidates = self.existed_word_neighbors(sentence[0])+[sentence[0]]
      if len(sentence) == 2:
        return [candidates[0], sentence[-1]]
      return [candidates[0]]
    cur_idx = 1
    first_candidates = self.existed_word_neighbors(sentence[0].lower())
    if len(first_candidates) == 0:
      first_candidates.append(sentence[0].lower())
    beam_sentences = [[self.p(word) if word!=sentence[0].lower() else 1, [word]] for idx, word in enumerate(first_candidates)]
    while cur_idx < len(sentence):
      candidates = self.existed_word_neighbors(sentence[cur_idx].lower())
      if len(candidates) == 0:
        candidates.append(sentence[cur_idx].lower())
      if not sentence[cur_idx].isalpha() or not sentence[cur_idx - 1].isalpha() or sentence[cur_idx].lower() in self.WORDS_FREQS or sentence[cur_idx - 1].lower() not in self.WORDS_FREQS or len(candidates) <= 1:
        for beam_sentence in beam_sentences:
          beam_sentence[1].append(sentence[cur_idx].lower())
        cur_idx += 1
        continue

      longer_beam_sentences = []

      for cand_idx, candidate in enumerate(candidates):
        for beam_sentence in beam_sentences:
          two_words_p = self.two_word_seq_prob(beam_sentence[1][-1].lower(), candidate.lower())
          longer_beam_sentences.append([beam_sentence[0] * self.p(candidate) * self.prob_lighting(two_words_p), beam_sentence[1]+[candidate]])
      # Normalizing probabilities
      s = 0
      for p, snt in longer_beam_sentences:
        s += p
      for i in range(len(longer_beam_sentences)):
        longer_beam_sentences[i][0] /= s

      beam_sentences = sorted(longer_beam_sentences, reverse=True)[:self.beam_search_N]
      cur_idx+=1
    return beam_sentences[0][1]

  def text_correction(self, text):
    sentences = self.tokenize(text)
    ans = ''
    for sentence in sentences:
        ans += ' '.join(self.sentence_correction(sentence))+' '
    return ans.strip()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [191]:
import re
from collections import Counter

class NorvigSpellChecker:

  def __init__(self):
    self.WORDS = Counter(self.words(open('big.txt').read()))

  def words(self, text):
     return re.findall(r'\w+', text.lower())

  def P(self, word):
    N=sum(self.WORDS.values())
    return self.WORDS[word] / N

  def correction(self, word):
    "Most probable spelling correction for word."
    return max(self.candidates(word), key=self.P)

  def candidates(self, word):
    "Generate possible spelling corrections for word."
    return (self.known([word]) or self.known(self.edits1(word)) or self.known(self.edits2(word)) or [word])

  def known(self, words):
    "The subset of `words` that appear in the dictionary of WORDS."
    return set(w for w in words if w in self.WORDS)

  def edits1(self, word):
    "All edits that are one edit away from `word`."
    letters    = 'abcdefghijklmnopqrstuvwxyz'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

  def edits2(self, word):
    "All edits that are two edits away from `word`."
    return (e2 for e1 in self.edits1(word) for e2 in self.edits1(e1))

  def text_correction(self, text):
    tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(text.lower())]
    ans = ''
    for sentence in tokenized_sentences:
      for word in sentence:
        nw = word
        if word.isalpha():
          nw = self.correction(word)
        ans += nw + ' '
    return ans

In [192]:
df = pd.read_csv("unigram_freq.csv") # https://www.kaggle.com/datasets/rtatman/english-word-frequency
df = df.dropna(axis=0)
WORDS = {}
for _, (word, count) in df.iloc[:10000].iterrows():
  WORDS[word.lower()] = count
print(WORDS['the'])

23135851162


In [193]:
f = pd.read_csv("bigrams.txt", delimiter='\t', header=None, encoding='latin1')
f = f.dropna(axis=0)
BIGRAMS = {}
for _, (count, word1, word2) in f.iterrows():
  BIGRAMS[(word1.lower(), word2.lower())] = count
print(BIGRAMS[('i', 'want')])

65359


## Justify your decisions

Write down justificaitons for your implementation choices. For example, these choices could be:
- Which ngram dataset to use
- Which weights to assign for edit1, edit2 or absent words probabilities
- Beam search parameters
- etc.

# Abstract
I chose Norvig's solution as the baseline model, but my model takes into account bigrams and utilizes beam search to maximize the probabilities of correction. My algorithm incorporates a simple notion: if a word is present in the dataset, then I do not attempt to correct it.

# Tokenization
I divide the text into sentences and then further segment these sentences into words using the `nltk` library.

# Word Probability
I obtain word counts from `unigram_freq.csv`, a dataset sourced from Kaggle ([link](https://www.kaggle.com/datasets/rtatman/english-word-frequency)). I utilize only the first 40,000 rows (the 40,000 most popular words) to slightly expedite the algorithm and exclude unfamiliar words such as 'evh' or 'sqv'. Although I am uncertain as to why these words are present in the dataset, the algorithm performs significantly better without them.

# Candidate Selection
I employ Norvig's solution, albeit with additional options. While Norvig's solution typically considers words within a two-edit distance, my algorithm includes an `edit_distance` parameter that can exceed 2.

# Incorporating Bigrams
I retrieve bigrams from the `bigrams.txt` file sourced from Moodle. If a particular bigram is absent from this file, I compute its probability as the product of the probabilities of its constituent words.

# Beam Search
For each word, the algorithm generates a list of correction candidates. Subsequently, I retain the `beam_search_N` most probable correction variants to correct the sentence (my algorithm processes text on a sentence-by-sentence basis). Then, for each pair consisting of the last word of the current sentence variant and the candidates for the current word, I calculate the probability using the bigram data. After each iteration of beam search, I decide whether to normalize the probabilities, as failure to do so would result in all probabilities diminishing too rapidly.


## Evaluate on a test set

Your task is to generate a test set and evaluate your work. You may vary the noise probability to generate different datasets with varying compexity. Compare your solution to the Norvig's corrector, and report the accuracies.

In [194]:
text = open("big.txt").read()
tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(text)]

In [195]:
N_sentences = 100
test_tokenized_sentences = random.sample(tokenized_sentences, N_sentences)

The following code implements a class that facilitates the construction of a dataset for evaluating a spell checker. This class takes text, the number of sentences to be sampled from the text, the noise level indicating how much the words will be changed, and the probability of a word being noisy.

In [196]:
import random

class SpellCheckerTestDatasetBuilder:
  def __init__(self, text, N_sentences=1, noise_level=1, noise_p=0.5):
    self.text = text
    self.tokenized_sentences = random.sample([word_tokenize(sentence) for sentence in sent_tokenize(text)], N_sentences)
    self.N_sentences=N_sentences
    self.noise_level = noise_level
    self.noise_p = noise_p
    self.noisy_tokenized_sentences = self.add_noise(self.tokenized_sentences)

  def get_noisy_text(self):
    ans = ''
    for sentence in self.noisy_tokenized_sentences:
      ans += ' '.join(sentence) + ' '
    return ans.strip()

  def add_noise_to_word(self, word):
    if not word.isalpha() or random.random() > self.noise_p:
      return word
    r = random.random()
    if 0 < r <= 0.25: # Insert
      random_position = random.randint(0, len(word))
      random_letter = chr(random.randint(97, 122))
      modified_word = word[:random_position] + random_letter + word[random_position:]
    if 0.25 < r <= 0.5: # Delete
      random_position = random.randint(0, len(word) - 1)
      modified_word = word[:random_position] + word[random_position + 1:]
    if 0.5 < r <= 0.75: # Replace
      random_position = random.randint(0, len(word) - 1)
      random_letter = chr(random.randint(97, 122))
      modified_word = word[:random_position] + random_letter + word[random_position + 1:]
    if 0.75 < r: # Transpose
      if len(word) < 2:
          return word
      random_position = random.randint(0, len(word) - 2)
      modified_word = list(word)
      modified_word[random_position], modified_word[random_position + 1] = modified_word[random_position + 1], modified_word[random_position]
      modified_word = ''.join(modified_word)
    return modified_word

  def noise_word(self, word):
    cur_word = word
    for i in range(self.noise_level):
      cur_word = self.add_noise_to_word(cur_word)
    return cur_word

  def add_noise(self, tokenized_sentences):
    noisy_sentences = []
    for sentence in tokenized_sentences:
      noisy_sentence = []
      for word in sentence:
        noisy_sentence.append(self.noise_word(word))
      noisy_sentences.append(noisy_sentence)
    return noisy_sentences

  def check_accuracy(self, text):
    c = 0
    N_words = 0
    for sent_idx, sentence in enumerate([word_tokenize(sentence) for sentence in sent_tokenize(text)]):
      for word_idx, word in enumerate(sentence):
        N_words += 1
        if sent_idx >=len(self.tokenized_sentences) or word_idx >= len(self.tokenized_sentences[sent_idx]):
          continue
        if word.lower() == self.tokenized_sentences[sent_idx][word_idx].lower():
          c += 1
    return c / N_words

In [211]:
# Building noisy dataset from 'big.txt' file
text = open('some_text_about_nature.txt').read()
test_dataset = SpellCheckerTestDatasetBuilder(text=text, N_sentences=10, noise_level=1, noise_p=1)
noisy_text = test_dataset.get_noisy_text()

In [212]:
print(noisy_text)

ehe rapdi increaseq i urbaniuzation as usepd mots orf teh revources liek treesc , mnerals , fodsil feuls , ad ater . Natur refevs t tue inteaction betxeen tje physcial surrouzndings aroundf su anfd tehe lifs wihin i lkie atmospherge , climae , natura resouces , ecosstem , ftlora , faun , nad huymans . Humanq ix thir quets fr ag comfortablx lviing hvae beepn sing hte resouces fo natuer mindlesly . Tqe soke dischargei fom actory untis nd exhauts tancks pof carb ys fcontaminating tht iar thgt woe bpeathe . Rgiht frmm he fdod wb emt , hte clothse ew war , asd te housie ew liv ni gis prvided yb naturte . Th us fo pestidides nad chemiccal lertilizers imn agricnlture adss tzo soi pollutoin . Ii js thj sprimary souurce ovf ell th necesisties bor teh nourishmnt xf ll lving beingsg gon Eahth . eW shouls conserv gwater yb is rationpal se , raipwater harvesing , chnecking tce sureace outfolow , ec . Ngature ii sacid o me tqe greatet teache a iyt tecahes hte lbessons f immogrtality an mortaliyt . h

In [215]:
splchk = SpellChecker(WORDS_FREQS=WORDS, BIGRAMS=BIGRAMS, edit_distance=2, beam_search_N=100)
norvigsp = NorvigSpellChecker()
corrected_text = splchk.text_correction(noisy_text)
corrected_text_norvig = norvigsp.text_correction(noisy_text)
print(f"My model: {test_dataset.check_accuracy(corrected_text)*100: .2f}% of text words corrected properly")
print(f"Norvig's model: {test_dataset.check_accuracy(corrected_text_norvig)*100: .2f}% of text words corrected properly")

My model:  71.42% of text words corrected properly
Norvig's model:  67.53% of text words corrected properly


## Models context understanding

In [214]:
splchk = SpellChecker(WORDS_FREQS=WORDS, BIGRAMS=BIGRAMS, edit_distance=2, beam_search_N=10)
norvigsp = NorvigSpellChecker()
first_text = "We sing ckngs!"
second_text = "We put on ckngs!"
print("Misspelled: "+first_text+"  | Corrected by my model:", splchk.text_correction(first_text), "Corrected by Norvig's model:", norvigsp.text_correction(first_text))
print("Misspelled: "+second_text+"| Corrected by my model:", splchk.text_correction(second_text), "Corrected by Norvig's model:", norvigsp.text_correction(second_text))

Misspelled: We sing ckngs!  | Corrected by my model: we sing songs ! Corrected by Norvig's model: we sing kings ! 
Misspelled: We put on ckngs!| Corrected by my model: we put on rings ! Corrected by Norvig's model: we put on kings ! 
